# Import package

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil,subprocess
import os
import subprocess
import re
import mzbatch
from pyopenms import MSExperiment,MzMLFile,PeakPickerHiRes
import time
import pyopenms
from pyopenms import Plotting
import importlib as imp
imp.reload(mzbatch)

In [1258]:
# Set path
root_path = '../../Fulaoshi/230821_mzML/'
middle_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']
batch_list = ['700_Amide','700_C18','873_Amide','873_C18','3545_Amide','3545_C18','BC_Amide','BC_C18']
out_root_path = '../../Fulaoshi/230821_mzML_res/rerun240410'

# Convert raw files to mzML

In [2]:
import subprocess

msconvert_path = 'msconvert'

for batch_path in batch_list:
    input_file = f'{root_path}/{batch_path}/*.raw'

    output_file = f'{out_root_path}/{batch_path}/'
    
    command = [msconvert_path, input_file, '--output', output_file]
    
    try:
        result = subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
    else:
        print("msconvert executed successfully.") 

# Process mzML files at 13 NCEs

In [1259]:
ref_all_df = pd.read_excel("../../Fulaoshi/230821_mzML/ref_all.xlsx",sheet_name="concat",index_col=0)
ref_all_df['batch_sampleid'] = ref_all_df['batch']+'/'+ref_all_df['sampleid']
ref_all_df['bscp']=ref_all_df['batch']+'+'+ref_all_df['sampleid'].astype(str)+'+'+ref_all_df['CAS No.']+'+'+ref_all_df['pos_neg']

In [1260]:
ref_all_df.head()

,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,M.Wt,pos_neg,refer_RT,m/z,RT time,NL,bscp,Adduct,batch_sampleid
0,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,pos,1,282.11960,196.2,117000000,873_Amide+26386+1867-73-8+pos,[M+H]+,873_Amide/26386
1,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,neg,1,280.10512,206.4,8250000,873_Amide+26386+1867-73-8+neg,[M-H]-,873_Amide/26386
2,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,346.46,pos,1,347.22160,63.6,25400000,873_Amide+26386+50-22-6+pos,[M+H]+,873_Amide/26386
3,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,346.46,neg,1,345.20713,63.6,3360000,873_Amide+26386+50-22-6+neg,[M-H]-,873_Amide/26386
4,873_Amide,Amide,26386,Hexylresorcinol,136-77-6,C12H18O2,194.27,neg,1,193.12340,57.0,27500000,873_Amide+26386+136-77-6+neg,[M-H]-,873_Amide/26386


In [1261]:
batch_sampleid_list = ref_all_df['batch_sampleid'].unique().tolist()
batch_sampleid_list[0:5],ref_all_df['batch_sampleid'].unique().shape

(['873_Amide/26386',
  'BC_Amide/S109',
  'BC_Amide/S111',
  '873_Amide/26387',
  '873_Amide/26388'],
 (348,))

In [1262]:
def get_filename_list(filepre_list):
    filename_list = []
    ev_list = ['1', '10', '100', '15', '20', '25', '30', '35', '40', '50', '60','70', '80']
    for filepre in filepre_list:
        for ev in ev_list:
            for tail in ['pos','neg']:
                filename_list.append(f'{filepre}-NCE{ev}-{tail}')
    return filename_list
filename_list = get_filename_list(['26386'])
str(filename_list)

"['26386-NCE1-pos', '26386-NCE1-neg', '26386-NCE10-pos', '26386-NCE10-neg', '26386-NCE100-pos', '26386-NCE100-neg', '26386-NCE15-pos', '26386-NCE15-neg', '26386-NCE20-pos', '26386-NCE20-neg', '26386-NCE25-pos', '26386-NCE25-neg', '26386-NCE30-pos', '26386-NCE30-neg', '26386-NCE35-pos', '26386-NCE35-neg', '26386-NCE40-pos', '26386-NCE40-neg', '26386-NCE50-pos', '26386-NCE50-neg', '26386-NCE60-pos', '26386-NCE60-neg', '26386-NCE70-pos', '26386-NCE70-neg', '26386-NCE80-pos', '26386-NCE80-neg']"

In [ ]:
for batch_sampleid in batch_sampleid_list[320:321]:
    bs_temp_df = ref_all_df[ref_all_df['batch_sampleid']==batch_sampleid]
    batch,sampleid = bs_temp_df.iloc[0,[0,2]]
    
    filename_list = get_filename_list([sampleid])
    
    pkl_root_path = f'{root_path}/{batch}/NCE_pkl/{sampleid}'
    if not os.path.exists(pkl_root_path):
        os.makedirs(pkl_root_path)
        
    compound_root_path = f'{out_root_path}/{batch}/NCE_compound/{sampleid}'
    if not os.path.exists(compound_root_path):
        os.makedirs(compound_root_path)
        
    log_record = pd.DataFrame(columns=['MS1_2dict_file','MS1_file','explain','refer_rt','limit_range'])
        
    print('Path:',batch_sampleid, sampleid, pkl_root_path)
    
    for filename in filename_list:
        filename_split_gang = filename.split('-') #['S1', 'NCE1', 'pos']
        pos_neg = filename_split_gang[2]
        ev = filename_split_gang[1]
        # print(pos_neg,ev)

        infile=f'{root_path}/{batch}/NCE2/{sampleid}/{filename}.mzML' #'../../Fulaoshi/230602_mzML/NCE2/S1/S1-NCE1-pos.mzML'
        print("input file：",infile)
        if not os.path.exists(infile):
            print('no this input file, continue ',infile)
            continue

        tic_pkl =f'{pkl_root_path}/{filename}.pkl' #'../../Fulaoshi/230602_mzML/NCE_pkl/S1/S1-NCE1-pos.pkl'
        tic_list=mzbatch.get_TIC_from_File(infile,tic_pkl)
        tic_df = pd.DataFrame(tic_list,columns=['ID','RT','Polarity','MSlevel','MS1_id','MS1_RT','pecursor'])
        tic_MS1_df = tic_df[tic_df['MSlevel'] == 1]
        tic_MS2_df = tic_df[tic_df['MSlevel'] == 2]
        
        temp_df = bs_temp_df[bs_temp_df["pos_neg"]==pos_neg]

        MS1_2dict_file=f'{pkl_root_path}/{filename}_MS1_2dict.pkl'
        MS1_2dict = mzbatch.make_MS1_data(tic_list,temp_df,tic_MS1_df,pos_neg,MS1_2dict_file)
        
        for temp_index in temp_df.index:
            compound_cas = temp_df.loc[temp_index,'CAS No.'] #1867-73-8
            mz = temp_df.loc[temp_index,'m/z']
            refer_rt = temp_df.loc[temp_index,'RT time']
            refer_rt_valid = temp_df.loc[temp_index,'refer_RT']
            # print(compound_cas, mz, refer_rt, refer_rt_valid)
            
            if (np.isnan(refer_rt) | np.isnan(mz)):
                # print(compound_cas, mz, refer_rt)
                continue
            if refer_rt_valid == -1:
                refer_rt = -1
                
            compound_folder = f'{compound_cas}_{ev}_{pos_neg}' #83207-58-3_NCE1_neg
            compound_folderpath = f'{compound_root_path}/{compound_folder}'
            if mzbatch.dir_is_exist(compound_folderpath):
                MS1_file = '{}/{}_MS1.csv'.format(compound_folderpath,compound_folder)

                MS1_data=MS1_2dict[compound_cas]
                pd.DataFrame(MS1_data).to_csv(MS1_file)

                [lower_limit_rt,upper_limit_rt],explain = mzbatch.get_rt_limit(MS1_data['rt'],MS1_data['intensity'],refer_rt)
                log_temp = pd.DataFrame({'MS1_2dict_file':[MS1_2dict_file],
                          'MS1_file':[MS1_file],'explain':[explain],
                          'refer_rt':[refer_rt],'limit_range':str([lower_limit_rt,upper_limit_rt])
                         })
                log_record = pd.concat([log_record, log_temp],ignore_index=True)

                if [lower_limit_rt,upper_limit_rt]==[-1,-1]:
                    continue
                temp_tic_MS2_df = tic_MS2_df[tic_MS2_df['pecursor'].apply(lambda x: mzbatch.less_5ppm(mzbatch.cal_ppm(mz,x)))]
                temp_tic_MS2_df = temp_tic_MS2_df[ (temp_tic_MS2_df['RT']>=lower_limit_rt) & (temp_tic_MS2_df['RT']<=upper_limit_rt) ]
                for temp_tic_MS2_index in temp_tic_MS2_df.index:
                    tic_ID = temp_tic_MS2_df.loc[temp_tic_MS2_index,'ID'] 
                    tic_RT = round(temp_tic_MS2_df.loc[temp_tic_MS2_index,'RT'],2)
                    MS2_file = '{}/{}_MS2_{}_{}.csv'.format(compound_folderpath,compound_folder,tic_ID,tic_RT)
                    
                    mz_uplimit = mzbatch.get_mz_uplimit(mz)
                    one_MS2_df = pd.DataFrame(tic_list[temp_tic_MS2_index]['peaks'],columns=['Mass','Intensity'])
                    one_MS2_df = one_MS2_df[one_MS2_df['Mass']<=mz_uplimit]
                    one_MS2_df.to_csv(MS2_file)


路径: 700_Amide/S90 S90 ../../Fulaoshi/230821_mzML//700_Amide/NCE_pkl/S90
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE1-pos.mzML
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE1-neg.mzML
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE10-pos.mzML
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE10-neg.mzML
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE100-pos.mzML
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE100-neg.mzML
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE15-pos.mzML
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE15-neg.mzML
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE20-pos.mzML
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE20-neg.mzML
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE25-pos.mzML
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE25-neg.mzML
输入文件： ../../Fulaoshi/230821_mzML//700_Amide/NCE2/S90/S90-NCE30-p

# Process mzML files at NCE 204060

In [ ]:
# Read reference data
ref_all_df = pd.read_excel("../../Fulaoshi/230821_mzML/ref_all.xlsx", sheet_name="third_1min", index_col=0)
ref_all_df['batch_sampleid'] = ref_all_df['batch'] + '/' + ref_all_df['sampleid']
ref_all_df.head()

,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,M.Wt,pos_neg,refer_RT,m/z,RT time,NL,bscp,batch_sampleid
0,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,pos,1,282.11960,196.2,117000000,873_Amide+26386+1867-73-8+pos,873_Amide/26386
1,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,neg,1,280.10512,206.4,8250000,873_Amide+26386+1867-73-8+neg,873_Amide/26386
2,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,346.46,pos,1,347.22160,63.6,25400000,873_Amide+26386+50-22-6+pos,873_Amide/26386
3,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,346.46,neg,1,345.20713,63.6,3360000,873_Amide+26386+50-22-6+neg,873_Amide/26386
4,873_Amide,Amide,26386,Hexylresorcinol,136-77-6,C12H18O2,194.27,neg,1,193.12340,57.0,27500000,873_Amide+26386+136-77-6+neg,873_Amide/26386


## Data correction

In [ ]:
mzML_path_list = [f'{root_path}/{batch}/NCE' for batch in batch_list]
for mzML_path in mzML_path_list:
    file_list = os.listdir(mzML_path)
    for filename in file_list:
        # S100-NCE204060-neg.mzML
        file_split_gang = filename[:-5].split('-')
        # 1. The name can only have 2 dashes
        if len(file_split_gang) != 3:
            print("two '-' ", mzML_path, filename)
        # 2. sampleid must exist in ref_all
        if file_split_gang[0] not in sampleid_list:
            print("no sampleid", mzML_path, filename)
        # 3. Energy must be preceded by NCE
        if not file_split_gang[1].startswith('NCE'):
            print("no NCE", mzML_path, filename)
        # break
    # break


sampleid没有 ../../Fulaoshi/230821_mzML/204060//873_C18/NCE 2_12-NCE204060-neg.mzML
sampleid没有 ../../Fulaoshi/230821_mzML/204060//873_C18/NCE 37-NCE204060-neg.mzML


In [ ]:
# Unique batch_sampleid_list
batch_sampleid_list = ref_all_df['batch_sampleid'].unique().tolist()
print(len(batch_sampleid_list))
batch_sampleid_list[0:5]


348


['873_Amide/26386',
 'BC_Amide/S109',
 'BC_Amide/S111',
 '873_Amide/26387',
 '873_Amide/26388']

In [ ]:
# Generate sampleid file list
def get_filename_list(filepre_list):
    filename_list = []
    ev_list = ['204060']
    for filepre in filepre_list:
        for ev in ev_list:
            for tail in ['pos', 'neg']:
                filename_list.append(f'{filepre}-NCE{ev}-{tail}')
    return filename_list

filename_list = get_filename_list(['26386'])
str(filename_list)


"['26386-NCE204060-pos', '26386-NCE204060-neg']"

In [ ]:
# Locate an mzML file using batch and sampleid
for batch_sampleid in batch_sampleid_list[320:321]:
    bs_temp_df = ref_all_df[ref_all_df['batch_sampleid'] == batch_sampleid]
    batch, sampleid = bs_temp_df.iloc[0, [0, 2]]
    
    # Generate filenames for positive/negative and energy based on sampleid
    filename_list = get_filename_list([sampleid])
    
    pkl_root_path = f'{root_path}/{batch}/NCE_pkl/{sampleid}'
    if not os.path.exists(pkl_root_path):
        os.makedirs(pkl_root_path)
        
    compound_root_path = f'{out_root_path}/{batch}/NCE_compound/{sampleid}'
    if not os.path.exists(compound_root_path):
        os.makedirs(compound_root_path)
        
    log_record = pd.DataFrame(columns=['MS1_2dict_file', 'MS1_file', 'explain', 'refer_rt', 'limit_range'])
        
    print('pathway:', batch_sampleid, sampleid, pkl_root_path)
    
    for filename in filename_list:
        filename_split_gang = filename.split('-')  # ['S1', 'NCE1', 'pos']
        pos_neg = filename_split_gang[2]
        ev = filename_split_gang[1]
        
        infile = f'{root_path}/{batch}/NCE/{filename}.mzML'  # '../../Fulaoshi/230602_mzML/NCE2/S1/S1-NCE1-pos.mzML'
        if not os.path.exists(infile):
            print('Input file not found, skipping ', infile)
            continue

        tic_pkl = f'{pkl_root_path}/{filename}.pkl'  # '../../Fulaoshi/230602_mzML/NCE_pkl/S1/S1-NCE1-pos.pkl'
        tic_list = mzbatch.get_TIC_from_File(infile, tic_pkl)
        tic_df = pd.DataFrame(tic_list, columns=['ID', 'RT', 'Polarity', 'MSlevel', 'MS1_id', 'MS1_RT', 'pecursor'])
        tic_MS1_df = tic_df[tic_df['MSlevel'] == 1]
        tic_MS2_df = tic_df[tic_df['MSlevel'] == 2]

        temp_df = bs_temp_df[bs_temp_df["pos_neg"] == pos_neg]
        
        MS1_2dict_file = f'{pkl_root_path}/{filename}_MS1_2dict.pkl'
        MS1_2dict = mzbatch.make_MS1_data(tic_list, temp_df, tic_MS1_df, pos_neg, MS1_2dict_file)
        
        for temp_index in temp_df.index:
            compound_cas = temp_df.loc[temp_index, 'CAS No.']  # 1867-73-8
            mz = temp_df.loc[temp_index, 'm/z']
            refer_rt = temp_df.loc[temp_index, 'RT time']
            refer_rt_valid = temp_df.loc[temp_index, 'refer_RT']
            
            if (np.isnan(refer_rt) | np.isnan(mz)):
                continue
            if refer_rt_valid == -1:
                refer_rt = -1
                
            compound_folder = f'{compound_cas}_{ev}_{pos_neg}'  # 83207-58-3_NCE1_neg
            compound_folderpath = f'{compound_root_path}/{compound_folder}'
            if mzbatch.dir_is_exist(compound_folderpath):
                MS1_file = '{}/{}_MS1.csv'.format(compound_folderpath, compound_folder)

                MS1_data = MS1_2dict[compound_cas]
                pd.DataFrame(MS1_data).to_csv(MS1_file)

                [lower_limit_rt, upper_limit_rt], explain = mzbatch.get_rt_limit(MS1_data['rt'], MS1_data['intensity'], refer_rt)
                log_temp = pd.DataFrame({'MS1_2dict_file': [MS1_2dict_file],
                                          'MS1_file': [MS1_file], 'explain': [explain],
                                          'refer_rt': [refer_rt], 'limit_range': str([lower_limit_rt, upper_limit_rt])})
                log_record = pd.concat([log_record, log_temp], ignore_index=True)

                if [lower_limit_rt, upper_limit_rt] == [-1, -1]:
                    continue
                temp_tic_MS2_df = tic_MS2_df[tic_MS2_df['pecursor'].apply(lambda x: mzbatch.less_5ppm(mzbatch.cal_ppm(mz, x)))]
                temp_tic_MS2_df = temp_tic_MS2_df[(temp_tic_MS2_df['RT'] >= lower_limit_rt) & (temp_tic_MS2_df['RT'] <= upper_limit_rt)]
                for temp_tic_MS2_index in temp_tic_MS2_df.index:
                    tic_ID = temp_tic_MS2_df.loc[temp_tic_MS2_index, 'ID']  # ID from the df, starting from 1
                    tic_RT = round(temp_tic_MS2_df.loc[temp_tic_MS2_index, 'RT'], 2)
                    MS2_file = '{}/{}_MS2_{}_{}.csv'.format(compound_folderpath, compound_folder, tic_ID, tic_RT)
                    pd.DataFrame(tic_list[temp_tic_MS2_index]['peaks'], columns=['Mass', 'Intensity']).to_csv(MS2_file)

    log_record_path = f'{out_root_path}/{batch}/NCE_log_record'
    if not os.path.exists(log_record_path):
        os.makedirs(log_record_path)
    while True:
        if os.path.exists(log_record_path):
            break
    log_record.to_csv(f'{log_record_path}/log_record_{sampleid}.csv')


路径: 700_Amide/S90 S90 ../../Fulaoshi/230821_mzML/204060//700_Amide/NCE_pkl/S90


# Optimization of isomer spectra


In [ ]:
# Read reference
ref_all_df = pd.read_excel("../../Fulaoshi/230821_mzML/ref_all.xlsx", sheet_name="concat", index_col=0)
ref_all_df['batch_sampleid'] = ref_all_df['batch'] + '/' + ref_all_df['sampleid']
ref_all_df['bscp'] = ref_all_df['batch'] + '+' + ref_all_df['sampleid'].astype(str) + '+' + ref_all_df['CAS No.'] + '+' + ref_all_df['pos_neg']


In [46]:
def get_bscpe_filename_list(bscp):
    ev_list = ['1', '10', '100', '15', '20', '25', '30', '35', '40', '50', '60','70', '80', '204060']
    return [f"{bscp}+{ev}" for ev in ev_list]
bscpe_list = get_bscpe_filename_list(bscp_list[219])

In [ ]:
ref_isomeride_df = pd.read_excel('../../Fulaoshi/230821_mzML/ref_isomers.xlsx', index_col=0)
ref_isomeride_df['bscp'] = ref_isomeride_df['batch'] + '+' + ref_isomeride_df['sampleid'].astype(str) + '+' + ref_isomeride_df['CAS No.'] + '+' + ref_isomeride_df['pos_neg']
ref_isomeride_df.head()


,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,M.Wt,pos_neg,refer_RT,m/z,RT time,min,max,NL,bscp
0,873_C18,C18,26393,2-(2-Methylbenzamido)acetic acid,42013-20-7,C10H11NO3,193.20,pos,1,194.08116,490.8,487.2,495.6,1.70E6,873_C18+26393+42013-20-7+pos
1,873_C18,C18,26393,2-(2-Phenylacetamido)acetic acid,500-98-1,C10H11NO3,193.20,pos,1,194.08116,474.6,470.4,480.0,7.41E6,873_C18+26393+500-98-1+pos
2,873_C18,C18,26396,L-Alloisoleucine,1509-34-8,C6H13NO2,131.17,pos,1,132.10190,118.2,109.8,123.6,7.02E6,873_C18+26396+1509-34-8+pos
3,873_C18,C18,26396,L-Leucine,61-90-5,C6H13NO2,131.17,pos,1,132.10190,133.8,127.2,140.4,5.80E6,873_C18+26396+61-90-5+pos
4,873_C18,C18,26397,5-Aminovaleric acid,660-88-8,C5H11NO2,117.15,pos,1,118.08625,54.0,51.0,57.0,2.79E7,873_C18+26397+660-88-8+pos


In [175]:
compound_dict = {}
for i in ref_isomeride_df.index:
    bscp,rt_min,rt_max = ref_isomeride_df.loc[i,["bscp","min","max"]]
    compound_dict[bscp] = [rt_min,rt_max]

In [ ]:
def del_folderpath_ms2files(compound_folderpath):
    compound_folderpath_files = os.listdir(compound_folderpath)
    compound_folderpath_ms2files = [i for i in compound_folderpath_files if "MS2" in i]
    for compound_folderpath_ms2file in compound_folderpath_ms2files:
        try:
            os.remove(f"{compound_folderpath}/{compound_folderpath_ms2file}")
        except Exception as e:
            print(f"Error deleting file {compound_folderpath_ms2file}: {e}")


In [ ]:
for bscp, [lower_limit_rt, upper_limit_rt] in compound_dict.items():
    mz = ref_isomeride_df[ref_isomeride_df["bscp"] == bscp].iloc[0]["m/z"]
    bscpe_list = get_bscpe_filename_list(bscp)
    print("===", bscp, mz, [lower_limit_rt, upper_limit_rt])
    
    for bscpe in bscpe_list:
        batch, sampleid, cas, pos_neg, ev = bscpe.split("+")
        filename = f"{sampleid}-NCE{ev}-{pos_neg}"
        if "204060" not in bscpe:
            infile = f'{root_path}/{batch}/NCE2/{sampleid}/{filename}.mzML'  # '../../Fulaoshi/230602_mzML/NCE2/S1/S1-NCE1-pos.mzML'
            pkl_root_path = f'{root_path}/{batch}/NCE_pkl/{sampleid}'
        else:
            infile = f'{root_path}/204060/{batch}/NCE/{filename}.mzML'  # '../../Fulaoshi/230602_mzML/NCE2/S1/S1-NCE1-pos.mzML'
            pkl_root_path = f'{root_path}/204060/{batch}/NCE_pkl/{sampleid}'
        
        compound_root_path = f'{out_root_path}/{batch}/NCE_compound/{sampleid}'
        compound_folder = f'{cas}_NCE{ev}_{pos_neg}'  # 83207-58-3_NCE1_neg
        compound_folderpath = f'{compound_root_path}/{compound_folder}'
        
        del_folderpath_ms2files(compound_folderpath)

        tic_pkl = f'{pkl_root_path}/{filename}.pkl'  # '../../Fulaoshi/230602_mzML/NCE_pkl/S1/S1-NCE1-pos.pkl'
        tic_list = mzbatch.get_TIC_from_File(infile, tic_pkl)
        tic_df = pd.DataFrame(tic_list, columns=['ID', 'RT', 'Polarity', 'MSlevel', 'MS1_id', 'MS1_RT', 'pecursor'])
        tic_MS2_df = tic_df[tic_df['MSlevel'] == 2]
        
        temp_tic_MS2_df = tic_MS2_df[tic_MS2_df['pecursor'].apply(lambda x: mzbatch.less_5ppm(mzbatch.cal_ppm(mz, x)))]
        temp_tic_MS2_df = temp_tic_MS2_df[(temp_tic_MS2_df['RT'] >= lower_limit_rt) & (temp_tic_MS2_df['RT'] <= upper_limit_rt)]
        
        if temp_tic_MS2_df.shape[0] == 0:
            print(bscpe, " no matches found")
            continue
        
        for temp_tic_MS2_index in temp_tic_MS2_df.index:
            tic_ID = temp_tic_MS2_df.loc[temp_tic_MS2_index, 'ID']  # ID from the df, starting from 1
            tic_RT = round(temp_tic_MS2_df.loc[temp_tic_MS2_index, 'RT'], 2)
            MS2_file = '{}/{}_MS2_{}_{}.csv'.format(compound_folderpath, compound_folder, tic_ID, tic_RT)
            
            mz_uplimit = mzbatch.get_mz_uplimit(mz)
            one_MS2_df = pd.DataFrame(tic_list[temp_tic_MS2_index]['peaks'], columns=['Mass', 'Intensity'])
            one_MS2_df = one_MS2_df[one_MS2_df['Mass'] <= mz_uplimit]
            one_MS2_df.to_csv(MS2_file)
            # print(MS2_file)

    # break


# Check that the log_record for each sampleid is present and not empty

In [ ]:
for batch_sampleid in batch_sampleid_list:
    batch, sampleid = batch_sampleid.split('/')
    
    log_record_file = f'{out_root_path}/{batch}/NCE_log_record/log_record_{sampleid}.csv'
    if not os.path.exists(log_record_file):
        print(f'Does not exist: {batch}/{sampleid} == ', batch_sampleid_list.index(f"{batch}/{sampleid}"), f' == log_record_{sampleid}.csv')
        continue
    one_log_record_df = pd.read_csv(f'{log_record_file}', index_col=0)
    if one_log_record_df.shape[0] == 0:
        print(f'Empty: {batch} == log_record_{sampleid}.csv')


不存在: 3545_C18/S47 ==  277  == log_record_S47.csv


# Merge the data of 13 NCEs and NCE 204060

In [ ]:
for batch in batch_list:
    print(batch)
    inpath = f'../../Fulaoshi/230821_mzML_res/204060/{batch}/NCE_compound/'
    sampleid_folder_list = os.listdir(inpath)
    for sampleid_folder in sampleid_folder_list:
        folder_inpath = f'{inpath}/{sampleid_folder}'
        com_folder_list = os.listdir(folder_inpath)
        for com_folder in com_folder_list:
            com_inpath = f'{folder_inpath}/{com_folder}'
            com_outpath = f'../../Fulaoshi/230821_mzML_res/rerun240410/{batch}/NCE_compound/{sampleid_folder}/{com_folder}'
            
            # Copy only if the 204060 file does not exist
            if not os.path.exists(com_outpath):
                shutil.copytree(com_inpath, com_outpath)
                # print(f'Does not exist: {com_outpath}')
            # Incorrect deletion code
            # if os.path.exists(com_outpath):
            #     shutil.rmtree(com_outpath)
                # print(com_outpath)
            # break
        # break
    # break


700_Amide
700_C18
873_Amide
873_C18
3545_Amide
3545_C18
BC_Amide
BC_C18
